# Lab J1 — Après-midi : Régression Linéaire
**Formation IA, Deep Learning & Machine Learning** — Julien Rolland
**Public :** M2 Développement Fullstack

---

## Objectifs

- Charger et explorer un dataset réel avec Pandas
- Implémenter la solution analytique des moindres carrés en NumPy
- Implémenter la descente de gradient from scratch
- Comparer les trois approches (analytique, GD, sklearn)


In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

print('NumPy  :', np.__version__)
print('Pandas :', pd.__version__)


---
## Partie 1 — Dataset

### 1.1 Chargement


In [ ]:
data = load_diabetes()

df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

print('Shape :', df.shape)
df.head()


In [ ]:
print(data.DESCR)

### 1.2 Exploration


In [ ]:
df.describe().round(2)


In [ ]:
print('NaN :', df.isna().sum().sum())


### 1.3 Visualisation — BMI vs target


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(df['bmi'], df['target'], alpha=0.4, s=15)
ax.set_xlabel('BMI (normalisé)')
ax.set_ylabel('Progression de la maladie')
ax.set_title('BMI vs Target')
plt.tight_layout()
plt.show()


### 1.4 Corrélations features / cible


In [ ]:
corr = df.drop(columns='target').corrwith(df['target']).sort_values()

fig, ax = plt.subplots(figsize=(7, 4))
colors = ['#d62728' if v < 0 else '#1f77b4' for v in corr]
ax.barh(corr.index, corr.values, color=colors)
ax.axvline(0, color='black', linewidth=0.8)
ax.set_xlabel('Corrélation de Pearson avec target')
ax.set_title('Corrélation features / cible')
plt.tight_layout()
plt.show()


---
## Partie 2 — Préparation des données

### 2.1 Train / test split


In [ ]:
X_raw = data.data          # (442, 10)
y_raw = data.target        # (442,)

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42
)

print('Train :', X_train_raw.shape, y_train.shape)
print('Test  :', X_test_raw.shape,  y_test.shape)


### 2.2 Ajout du biais

Pour absorber le terme constant $b$ dans le vecteur de poids $\mathbf{w}$, on augmente $X$ d'une colonne de 1 :

$$\tilde{X} = [\mathbf{1} \mid X], \quad \tilde{\mathbf{w}} = [b, w_1, \dots, w_d]^\top$$

$$\hat{y} = \tilde{X}\, \tilde{\mathbf{w}}$$


In [ ]:
def add_bias(X: np.ndarray) -> np.ndarray:
    """Prepend a column of ones to X.  Shape: (N, d) -> (N, d+1)."""
    raise NotImplementedError

### 2.3 Normalisation

La descente de gradient est sensible à l'échelle des features : si une feature varie de 0 à 1000 et une autre de 0 à 1, leurs gradients ont des magnitudes très différentes et le même $\alpha$ est trop grand pour l'une, trop petit pour l'autre.

On centre-réduit chaque feature avec les statistiques du **train set** uniquement — utiliser les stats du test set constituerait une fuite de données (*data leakage*) :

$$\tilde{x}_j = \frac{x_j - \mu_j}{\sigma_j}$$

La colonne de biais (tout à 1) est ajoutée **après** normalisation : elle n'a pas à être normalisée.

In [ ]:
X_mean = X_train_raw.mean(axis=0)
X_std  = X_train_raw.std(axis=0)

X_train_norm = add_bias((X_train_raw - X_mean) / X_std)
X_test_norm  = add_bias((X_test_raw  - X_mean) / X_std)

print('X_train_norm shape :', X_train_norm.shape)
print('X_test_norm  shape :', X_test_norm.shape)

---
## Définition du modèle

Deux fonctions réutilisées dans toutes les parties.

**Prédiction linéaire**

$$\hat{y} = X\mathbf{w}$$

**Erreur quadratique moyenne (MSE)**

$$\mathcal{L} = \frac{1}{N}\sum_{i=1}^{N}(y_i - \hat{y}_i)^2 = \frac{1}{N}\|y - \hat{y}\|^2$$

In [ ]:
def predict(X: np.ndarray, w: np.ndarray) -> np.ndarray:
    """
    Prédiction linéaire.

    Parameters
    ----------
    X : ndarray of shape (N, d)
        Matrice de features avec colonne de biais prepended.
    w : ndarray of shape (d,)
        Vecteur de poids (biais inclus).

    Returns
    -------
    y_hat : ndarray of shape (N,)
        Valeurs prédites.
    """
    raise NotImplementedError


def mse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Erreur quadratique moyenne.

    Parameters
    ----------
    y_true : ndarray of shape (N,)
        Valeurs cibles réelles.
    y_pred : ndarray of shape (N,)
        Valeurs prédites.

    Returns
    -------
    float
        (1/N) * ||y_true - y_pred||²
    """
    raise NotImplementedError

---
## Partie 3 — Solution Analytique

La solution des moindres carrés minimise $\|y - Xw\|^2$ :

$$\mathbf{w}^* = (X^\top X)^{-1} X^\top y$$

En pratique on résout le système linéaire $(X^\top X)\,\mathbf{w} = X^\top y$ avec `np.linalg.solve` (plus stable que l'inversion explicite).

### 3.1 Implémentation


In [ ]:
def analytical_solution(X: np.ndarray, y: np.ndarray) -> np.ndarray:
    """
    Régression linéaire par résolution analytique (OLS).

    Parameters
    ----------
    X : ndarray of shape (N, d)
        Matrice de features avec colonne de biais prepended.
    y : ndarray of shape (N,)
        Valeurs cibles.

    Returns
    -------
    w : ndarray of shape (d,)
        Vecteur de poids optimal.
    """
    raise NotImplementedError

### 3.2 Entraînement et évaluation


In [ ]:
w_ols = analytical_solution(X_train_norm, y_train)

y_pred_train_ols = predict(X_train_norm, w_ols)
y_pred_test_ols  = predict(X_test_norm,  w_ols)

mse_train_ols = mse(y_train, y_pred_train_ols)
mse_test_ols  = mse(y_test,  y_pred_test_ols)

print(f'OLS — MSE train : {mse_train_ols:.2f}')
print(f'OLS — MSE test  : {mse_test_ols:.2f}')

### 3.3 Prédictions vs réalité


In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
lims = [y_test.min(), y_test.max()]
ax.scatter(y_test, y_pred_test_ols, alpha=0.5, s=20)
ax.plot(lims, lims, 'r--', linewidth=1, label='y = ŷ')
ax.set_xlabel('Valeurs réelles')
ax.set_ylabel('Prédictions OLS')
ax.set_title('Solution analytique — test set')
ax.legend()
plt.tight_layout()
plt.show()


---
## Partie 4 — Descente de Gradient

### 4.1 Implémentation

La mise à jour du gradient pour MSE est :

$$\mathbf{w} \leftarrow \mathbf{w} - \alpha \cdot \frac{2}{N} X^\top (X\mathbf{w} - y)$$

In [ ]:
def gradient_descent(
    X: np.ndarray,
    y: np.ndarray,
    lr: float = 0.1,
    n_iters: int = 1000,
) -> tuple:
    """
    Gradient descent for linear regression.

    Returns
    -------
    w       : final weight vector, shape (d,)
    history : MSE at each iteration, shape (n_iters,)
    """
    N, d = X.shape
    w = np.zeros(d)
    history = np.empty(n_iters)

    for i in range(n_iters):
        raise NotImplementedError

    return w, history

### 4.2 Entraînement

In [ ]:
w_gd, history = gradient_descent(X_train_norm, y_train, lr=0.1, n_iters=1000)

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(history)
ax.set_yscale('log')
ax.set_xlabel('Itération')
ax.set_ylabel('MSE (log scale)')
ax.set_title('Courbe d\'apprentissage — Descente de gradient')
plt.tight_layout()
plt.show()

### 4.3 Évaluation

In [ ]:
y_pred_train_gd = predict(X_train_norm, w_gd)
y_pred_test_gd  = predict(X_test_norm,  w_gd)

mse_train_gd = mse(y_train, y_pred_train_gd)
mse_test_gd  = mse(y_test,  y_pred_test_gd)

print(f'GD — MSE train : {mse_train_gd:.2f}')
print(f'GD — MSE test  : {mse_test_gd:.2f}')

---
## Partie 5 — Comparaison avec scikit-learn

### 5.1 LinearRegression


In [ ]:
model = LinearRegression()
model.fit(X_train_norm[:, 1:], y_train)

y_pred_train_sk = model.predict(X_train_norm[:, 1:])
y_pred_test_sk  = model.predict(X_test_norm[:, 1:])

mse_train_sk = mse(y_train, y_pred_train_sk)
mse_test_sk  = mse(y_test,  y_pred_test_sk)

print(f'sklearn — MSE train : {mse_train_sk:.2f}')
print(f'sklearn — MSE test  : {mse_test_sk:.2f}')

### 5.2 Tableau comparatif


In [ ]:
results = pd.DataFrame({
    'Méthode'   : ['Analytique (OLS)', 'Gradient Descent', 'sklearn'],
    'MSE train' : [mse_train_ols, mse_train_gd, mse_train_sk],
    'MSE test'  : [mse_test_ols,  mse_test_gd,  mse_test_sk],
}).set_index('Méthode').round(2)

results


### 5.3 Comparaison des coefficients appris


In [ ]:
coef_ols = w_ols[1:]
coef_gd  = w_gd[1:]
coef_sk  = model.coef_

feature_names = data.feature_names
x_pos = np.arange(len(feature_names))
width = 0.25

fig, ax = plt.subplots(figsize=(10, 4))
ax.bar(x_pos - width, coef_ols, width, label='Analytique (OLS)')
ax.bar(x_pos,         coef_gd,  width, label='Gradient Descent', alpha=0.8)
ax.bar(x_pos + width, coef_sk,  width, label='sklearn', alpha=0.8)
ax.set_xticks(x_pos)
ax.set_xticklabels(feature_names, rotation=45, ha='right')
ax.axhline(0, color='black', linewidth=0.8)
ax.set_ylabel('Coefficient')
ax.set_title('Coefficients appris — OLS vs GD vs sklearn')
ax.legend()
plt.tight_layout()
plt.show()

---
## Conclusion

| Méthode | MSE train | MSE test | Remarque |
|---|---|---|---|
| Analytique (OLS) | 2868.55 | 2900.19 | Solution exacte, $O(d^3)$ |
| Descente de gradient | 2869.33 | 2895.30 | Itératif, scalable |
| sklearn | 2868.55 | 2900.19 | Référence |

> Les trois méthodes convergent vers la même solution : la **descente de gradient est la clé de tout le deep learning**. En passant d'une fonction de perte convexe (MSE) à des réseaux profonds non-convexes, le même algorithme reste au cœur de l'optimisation — complété par la rétropropagation pour calculer les gradients.


---
## Pour aller plus loin — Régression non-linéaire par transformation de features

La régression linéaire ne peut fitter qu'un hyperplan : $\hat{y} = \mathbf{w}^\top \mathbf{x}$.

Si la relation entre $x$ et $y$ est non-linéaire, il suffit de créer un **nouveau dataset** en appliquant une transformation $\varphi$ à chaque observation, puis de faire tourner **exactement le même algorithme** dessus.

$$\varphi(\mathbf{x}) = [1,\ x_1,\ x_1^2,\ \dots,\ x_1^d,\ x_2,\ x_2^2,\ \dots,\ x_2^d,\ \ldots]$$

Le modèle reste linéaire en les paramètres $\mathbf{w}$, mais devient non-linéaire en l'entrée $\mathbf{x}$.

In [ ]:
def phi(X: np.ndarray, degree: int) -> np.ndarray:
    """
    Expansion polynomiale par feature.

    Pour chaque colonne de X, génère les puissances x, x², ..., x^degree.
    Ajoute une colonne de biais en tête.

    Parameters
    ----------
    X : ndarray of shape (N, d)
        Matrice de features (sans biais).
    degree : int
        Degré maximal.

    Returns
    -------
    X_phi : ndarray of shape (N, 1 + d * degree)
        Features transformées avec biais.
    """
    cols = [np.ones(len(X))]
    for j in range(X.shape[1]):
        for p in range(1, degree + 1):
            cols.append(X[:, j] ** p)
    return np.stack(cols, axis=1)

### Exemple — expansion de degré 2 sur BMI et s5

BMI et s5 sont les deux features les plus corrélées avec la cible (cf. 1.4).

In [ ]:
# BMI = col 3, s5 = col 9 dans X_train_norm (après biais)
X_sel_train = X_train_norm[:, [3, 9]]
X_sel_test  = X_test_norm[:, [3, 9]]

degree = 2
X_phi_train = phi(X_sel_train, degree)
X_phi_test  = phi(X_sel_test,  degree)

print(f'Shape avant φ : {X_sel_train.shape}  →  après φ : {X_phi_train.shape}')

model_phi = LinearRegression(fit_intercept=False)
model_phi.fit(X_phi_train, y_train)

mse_train_phi = mse(y_train, model_phi.predict(X_phi_train))
mse_test_phi  = mse(y_test,  model_phi.predict(X_phi_test))

print(f'φ(BMI, s5, d={degree}) — MSE train : {mse_train_phi:.2f} | MSE test : {mse_test_phi:.2f}')
print(f'Baseline linéaire (10 features)  — MSE test  : {mse_test_ols:.2f}')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
lims = [y_test.min(), y_test.max()]
ax.scatter(y_test, model_phi.predict(X_phi_test), alpha=0.5, s=20)
ax.plot(lims, lims, 'r--', linewidth=1, label='y = ŷ')
ax.set_xlabel('Valeurs réelles')
ax.set_ylabel('Prédictions φ')
ax.set_title(f'φ(BMI, s5, d={degree}) — test set')
ax.legend()
plt.tight_layout()
plt.show()

**À vous de jouer !** Pouvez-vous réduire le MSE test ?

Quelques pistes :
- Changer le degré du polynôme
- Utiliser plusieurs features à la fois (pas seulement le BMI)
- Combiner des features entre elles ($x_\text{bmi} \times x_\text{bp}$, ...)